In [1]:
!pip install -Uq wikipedia

  Preparing metadata (setup.py) ... done


In [2]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin, unquote
import requests
import pandas as pd
import re
from fastcore.all import *
from random import randint
import wikipedia

In [6]:
url = 'https://or.wikipedia.org/wiki/%E0%AC%AC%E0%AC%BF%E0%AC%B6%E0%AD%87%E0%AC%B7:%E0%AC%B8%E0%AC%AC%E0%AD%81%E0%AC%AA%E0%AD%83%E0%AC%B7%E0%AD%8D%E0%AC%A0%E0%AC%BE'
base_url = 'or.wikipedia.org'
resp = requests.get(url, timeout=10)
soup = BeautifulSoup(resp.text, 'html.parser')

In [10]:
def nxt_page_href(soup):
  try:
    hrefs = list(set([a_tag.get('href') for a_tag in soup.findAll('a') if 'ପର ପୃଷ୍ଠା' in a_tag.get_text()]))
    return hrefs[0] if hrefs else None
  except Exception as e:
    print(e)
    return None
nxt_page_href(soup)

'/w/index.php?title=%E0%AC%AC%E0%AC%BF%E0%AC%B6%E0%AD%87%E0%AC%B7:%E0%AC%B8%E0%AC%AC%E0%AD%81%E0%AC%AA%E0%AD%83%E0%AC%B7%E0%AD%8D%E0%AC%A0%E0%AC%BE&from=1306'

In [12]:
full_url = lambda end: urljoin("https://"+ base_url, unquote(end))
full_url(nxt_page_href(soup))

'https://or.wikipedia.org/w/index.php?title=ବିଶେଷ:ସବୁପୃଷ୍ଠା&from=1306'

In [14]:
def all_pages(soup):
  div_tag = soup.find("div", class_="mw-allpages-body")
  if not div_tag:
    return None
  lin = [full_url(a.get('href')) for a in div_tag.findAll('a')]
  return lin if lin else None
type(all_pages(soup)) == list

True

In [15]:
p_count = 0
content_links = []
while True:
  p_count += 1
  resp = requests.get(url)
  soup = BeautifulSoup(resp.text, 'html.parser')

  content_links += all_pages(soup) if all_pages(soup) else []

  nxt_href = nxt_page_href(soup)
  if not nxt_href :
    break
  url = full_url(nxt_href)

url, p_count, len(content_links)


('https://or.wikipedia.org/w/index.php?title=ବିଶେଷ:ସବୁପୃଷ୍ଠା&from=୯+ଜୁଲାଇ',
 111,
 38111)

In [16]:
# prompt: create a dataframe from list and save it to csv

import pandas as pd

# Assuming 'content_links' is your list
df = pd.DataFrame({'links': content_links})

# Save the DataFrame to a CSV file
df.to_csv('content_links.csv', index=False)


In [17]:
!ls

content_links.csv  sample_data
